In [24]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.engine.topology import Input
from keras.layers import concatenate,Conv2DTranspose
from keras.models import Model
import numpy as np
import random
from keras import backend as K
smooth = 1
import cv2
import os
import pickle


Start


In [11]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


Saving seg_data.pkl to seg_data (2).pkl
User uploaded file "seg_data.pkl" with length 196851436 bytes


In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


data = []
labels = []

d = pickle.load(open( 'seg_data (2).pkl', 'rb'))
data = d['data']
labels = d['labels']


loading images...
Dones.


In [14]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels, dtype="float") / 255.0

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)


Done


In [29]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

  
batch_size = 64
num_classes = 2
epochs = 20

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

inputs = Input((img_rows, img_cols, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

#model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Score:', score[1])

Train on 1600 samples, validate on 400 samples
Epoch 1/20
 - 9s - loss: -2.7696e-01 - dice_coef: 0.2770 - val_loss: -2.8729e-01 - val_dice_coef: 0.2873
Epoch 2/20
 - 7s - loss: -2.7750e-01 - dice_coef: 0.2775 - val_loss: -2.8827e-01 - val_dice_coef: 0.2883
Epoch 3/20
 - 7s - loss: -2.7893e-01 - dice_coef: 0.2789 - val_loss: -2.8996e-01 - val_dice_coef: 0.2900
Epoch 4/20
 - 7s - loss: -2.8100e-01 - dice_coef: 0.2810 - val_loss: -2.9341e-01 - val_dice_coef: 0.2934
Epoch 5/20
 - 7s - loss: -2.8788e-01 - dice_coef: 0.2879 - val_loss: -3.0879e-01 - val_dice_coef: 0.3088
Epoch 6/20
 - 7s - loss: -3.2312e-01 - dice_coef: 0.3231 - val_loss: -3.5305e-01 - val_dice_coef: 0.3530
Epoch 7/20
 - 7s - loss: -3.4229e-01 - dice_coef: 0.3423 - val_loss: -3.5911e-01 - val_dice_coef: 0.3591
Epoch 8/20
 - 7s - loss: -3.5211e-01 - dice_coef: 0.3521 - val_loss: -3.7093e-01 - val_dice_coef: 0.3709
Epoch 9/20
 - 7s - loss: -3.7309e-01 - dice_coef: 0.3731 - val_loss: -3.9890e-01 - val_dice_coef: 0.3989
Epoch 10